In [28]:
import numpy as np
import matplotlib.pyplot as plt

from spectools import lpsd, dsp
lpsd.use_c_core = False

import multiprocessing as mp
ctx = mp.get_context('fork')
pool = ctx.Pool()

In [29]:
size = int(1e5)
rho = 1e-3
pink_x = np.fft.irfft(np.fft.rfft(np.random.normal(size=size)) / np.sqrt(np.arange(size//2+1)+1))
pink_y = rho * pink_x + np.sqrt(1 - rho**2) * np.fft.irfft(np.fft.rfft(np.random.normal(size=size)) / np.sqrt(np.arange(size//2+1)+1))
brown_noise_1 = np.cumsum(np.random.normal(0, 1, size))
brown_noise_2 = np.cumsum(np.random.normal(0, 1, size))
signal_1 = pink_x + 1*np.sin(2*np.pi*2.12345e-3*np.arange(size) + 1e-3*brown_noise_1) + 1*np.sin(2*np.pi*1e-1*np.arange(size) + 1e-3*brown_noise_2)
signal_2 = pink_y + (1/3)*np.sin(2*np.pi*2.12345e-3*np.arange(size) + 2e-3*brown_noise_1) + (1/3)*np.sin(2*np.pi*1e-1*np.arange(size) + 1e-3*brown_noise_2)

In [ ]:
fig, ax = plt.subplots(figsize=(6,2), dpi=150)

ax.plot(signal_1, lw=1, c='k')
ax.plot(signal_2, lw=1, c='orange', alpha=0.5)
# ax.set_xlim(0,500)
ax.set_xlabel('Sample')
ax.set_ylabel('Signal')

Let's compute the ASD of `pink_x` and `pink_y` while passing the multiprocessing.Pool argument, which allows parallelizing the execution of the LPSD algorithm across the scheduled frequencies.

In [31]:
f, asd_x = lpsd.asd(signal_1, 1, pool=pool, bmin=8)
_, asd_y = lpsd.asd(signal_2, 1, pool=pool, bmin=8)

f, ps_x = lpsd.ps(signal_1, 1, pool=pool, bmin=8)
_, ps_y = lpsd.ps(signal_2, 1, pool=pool, bmin=8)

In [ ]:
figsize=(3,2.5)
dpi=300
fontsize=8
linewidth=1

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.loglog(f, asd_x, linewidth=linewidth, color="black");
ax.loglog(f, asd_y, linewidth=linewidth, color="orange");
ax.set_xlim([f[0], f[-1]])
ax.set_xlabel("Fourier frequency (Hz)", fontsize=fontsize);
ax.set_ylabel(r"Amplitude spectral density", fontsize=fontsize);
ax.tick_params(axis='both', which='both', labelsize=fontsize)
ax.grid();
fig.tight_layout();

In [ ]:
figsize=(3,2.5)
dpi=300
fontsize=8
linewidth=1

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.loglog(f, ps_x, linewidth=linewidth, color="black");
ax.loglog(f, ps_y, linewidth=linewidth, color="orange");
ax.set_xlim([f[0], f[-1]])
ax.set_ylim(1e-2,1)
ax.set_xlabel("Fourier frequency (Hz)", fontsize=fontsize);
ax.set_ylabel(r"Power spectrum", fontsize=fontsize);
ax.tick_params(axis='both', which='both', labelsize=fontsize)
ax.grid(which='both');
fig.tight_layout();

Now, let's compute the CSD and coherence of `pink_x` and `pink_y`

In [34]:
f, csd = lpsd.csd([signal_1,signal_2], 1, pool=pool)
_, cf = lpsd.coh([signal_1,signal_2], 1, pool=pool)
_, coh = lpsd.coh([signal_1,signal_2], 1, pool=pool)

In [ ]:
figsize=(3,4)
dpi=300
fontsize=8
linewidth=1

fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=figsize, dpi=dpi, gridspec_kw={'height_ratios': [1, 1, 1]})
ax1.semilogx(f, coh, linewidth=linewidth, label="test", color="black");
ax2.loglog(f, np.abs(csd), linewidth=linewidth, label="test", color="black");
ax3.loglog(f, cf, linewidth=linewidth, label="test", color="black");
ax3.set_xlabel("Fourier frequency (Hz)", fontsize=fontsize);
ax1.set_ylabel("Coherence", fontsize=fontsize);
ax2.set_ylabel("CSD magnitude", fontsize=fontsize);
ax3.set_ylabel("Coupling coefficient", fontsize=fontsize);
ax1.set_xlim([f[0], f[-1]])
ax2.set_xlim([f[0], f[-1]])
ax3.set_xlim([f[0], f[-1]])
ax1.tick_params(which='both', labelsize=fontsize);
ax2.tick_params(which='both', labelsize=fontsize);
ax3.tick_params(which='both', labelsize=fontsize);
ax1.grid();
ax2.grid();
ax3.grid();
fig.tight_layout();
fig.align_ylabels();


The previous functions returned the Fourier frequency array and the desired measurement array (e.g., the ASD, CSD, coherence, etc). In the more general workflow, an LTFObject is returned, which contains attributes of all possible measurements. 

In [36]:
csd_obj = lpsd.ltf((signal_1, signal_2), 1, pool=pool)
csd_obj_inband = lpsd.ltf((signal_1, signal_2), 1, band=[1e-3,1e-1], pool=pool)

In [ ]:
csd_obj_inband.f

In [ ]:
figsize=(3,2.5)
dpi=300
fontsize=8
linewidth=1

fig, (ax1, ax2) = plt.subplots(2,1, figsize=figsize, dpi=dpi, gridspec_kw={'height_ratios': [1, 1]})
ax1.semilogx(f, csd_obj.coh, linewidth=linewidth, label="test", color="k", ls='-');
ax1.semilogx(csd_obj_inband.f, csd_obj_inband.coh, linewidth=linewidth, label="test", color="orange", ls='--');
ax2.loglog(f, np.abs(csd_obj.Hxy), linewidth=linewidth, label="test", color="k", ls='-');
ax2.loglog(csd_obj_inband.f, np.abs(csd_obj_inband.Hxy), linewidth=linewidth, label="test", color="orange", ls='--');
ax2.set_xlabel("Fourier frequency (Hz)", fontsize=fontsize);
ax1.set_ylabel("Coherence", fontsize=fontsize);
ax2.set_ylabel("abs(CSD)", fontsize=fontsize);
ax1.set_xlim([f[0], f[-1]])
ax2.set_xlim([f[0], f[-1]])
ax1.tick_params(which='both', labelsize=fontsize);
ax2.tick_params(which='both', labelsize=fontsize);
ax1.grid();
ax2.grid();
fig.tight_layout();
fig.align_ylabels();


The `LTFObject` class has a generic plot method for a variety of measurements... plot options can be passed as optional arguments

In [ ]:
csd_obj.plot(which='cs')

In [ ]:
csd_obj.plot(which='cs', c='orange', lw=2, ls='-')

In [ ]:
csd_obj.plot(which='coh', color='tomato', lw=1, ls='-', ylabel='Coherence function')

The `get_measurement` method allows estimating a measurement at a specified frequency by interpolation of the existing measurements. For example, let's compute the coupling coefficient precisely at 1 mHz:

In [ ]:
csd_obj.get_measurement(1e-3, which='cf')

Apart from the spectral estimates, the target measurement could be the bin number `m` or the frequency resolution `r`: 

In [ ]:
csd_obj.get_measurement(1e-3, which='m')

In [ ]:
csd_obj.get_measurement(1e-3, which='r')

In [ ]:
for f, r, m in zip(csd_obj.f, csd_obj.r, csd_obj.m):
    print(f"{f:6.6f}, {r:6.6f}, {m:6.3f}")

We can also perform single-bin computations

In [ ]:
csd_obj.get_measurement(1e-3, which='r')

In [47]:
csd_obj2 = lpsd.ltf_single_bin([pink_x, pink_y], 1, 1e-3, csd_obj.get_measurement(1e-3, which='r'))

In [ ]:
csd_obj2.cf, csd_obj.get_measurement(1e-3, which='cf')

In [49]:
coh = []
cf = []

for f, fres in zip(csd_obj.f, csd_obj.r):
    csd_sb = lpsd.ltf_single_bin([signal_1, signal_2], 1, f, fres)
    cf.append(csd_sb.cf)
    coh.append(csd_sb.coh)

In [ ]:
figsize=(3,2)
dpi=300
fontsize=8
linewidth=1.5

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.loglog(csd_obj.f, csd_obj.cf, linewidth=linewidth, label="test", color="black");
ax.loglog(csd_obj.f, cf, linewidth=linewidth, label="test", color="orange", ls='--');
ax.set_xlim([csd_obj.f[0],csd_obj.f[-1]])
ax.set_xlabel("Fourier frequency (Hz)", fontsize=fontsize);
ax.tick_params(axis='both', which='both', labelsize=fontsize)
ax.grid();
fig.tight_layout();

In [ ]:
figsize=(3,2)
dpi=300
fontsize=8
linewidth=1.

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.loglog(csd_obj.f, np.abs((csd_obj.cf-cf)/csd_obj.cf), linewidth=linewidth, label="test", color="red", ls='-');
ax.set_xlim([csd_obj.f[0],csd_obj.f[-1]])
ax.set_xlabel("Fourier frequency (Hz)", fontsize=fontsize);
ax.tick_params(axis='both', which='both', labelsize=fontsize)
ax.grid();
fig.tight_layout();

Let's now compute the auto-spectrum of `pink_x` in LTFObject format:

In [52]:
psd_obj = lpsd.ltf(pink_x, 1, pool=pool)

The RMS signal variations from the ASD measurement is calculated via the `get_rms` method:

In [ ]:
psd_obj.get_rms()

We can generate a timeseries from a calculated PSD via the `get_timeseries` method:

In [ ]:
timeseries = psd_obj.get_timeseries(0.5, 1e6)

In [ ]:
fig, ax = plt.subplots(figsize=(6,2), dpi=150)
ax.plot(timeseries, c='k', lw=1)
ax.set_xlabel('Sample')
ax.set_ylabel('Signal')

In [ ]:
psd_obj2 = lpsd.ltf(timeseries, 1, pool=pool)

In [ ]:
figsize=(3,2.5)
dpi=300
fontsize=8
linewidth=2

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.loglog(psd_obj.f, psd_obj.asd, linewidth=linewidth, color="black");
ax.loglog(psd_obj2.f, psd_obj2.asd, linewidth=1, color="orange", ls='-');
ax.set_xlim([psd_obj.f[0], psd_obj.f[-1]])
ax.set_xlabel("Fourier frequency (Hz)", fontsize=fontsize);
ax.set_ylabel(r"ASD $\rm (A/Hz^{1/2})$", fontsize=fontsize);
ax.tick_params(axis='both', which='both', labelsize=fontsize)
ax.grid();
fig.tight_layout();